# Table of Contents
 <p><div class="lev1"><a href="#Lea-package-first-studies"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lea package first studies</a></div><div class="lev2"><a href="#Flip-coin"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Flip coin</a></div><div class="lev2"><a href="#&quot;Rain-Sprinkler-Grass&quot;-bayesian-network"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>"Rain-Sprinkler-Grass" bayesian network</a></div><div class="lev2"><a href="#References"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>References</a></div>

# Lea package first studies

## Flip coin

In [2]:
from lea import *

In [4]:
flip1 = Lea.fromValFreqs(('head',67),('tail',33))
flip1

head : 67/100
tail : 33/100

In [5]:
P(flip1=='head')

67/100

In [6]:
Pf(flip1=='head')

0.67

In [9]:
flip1.random(10)

('head',
 'tail',
 'head',
 'tail',
 'head',
 'head',
 'head',
 'head',
 'tail',
 'tail')

In [10]:
flip2 = flip1.clone()
flips = flip1 + '-' + flip2
flips

head-head : 4489/10000
head-tail : 2211/10000
tail-head : 2211/10000
tail-tail : 1089/10000

In [11]:
P(flip1==flip2)

2789/5000

In [13]:
P(flip1!=flip2)

2211/5000

In [15]:
flip1.upper()
flip1.upper()[0]

H : 67/100
T : 33/100

In [16]:
def toInt(flip):
    return 1 if flip=='head' else 0

headCount1 = flip1.map(toInt)
headCount1

0 : 33/100
1 : 67/100

In [17]:
headCount2 = flip2.map(toInt)
headCounts = headCount1 + headCount2
headCounts 

0 : 1089/10000
1 : 4422/10000
2 : 4489/10000

In [20]:
headCounts.given(flip1==flip2)
headCounts.given(flip1!=flip2)
flip1.given(headCounts==0)
flip1.given(headCounts==1)

flip1.given(headCounts==2)

head : 1

## "Rain-Sprinkler-Grass" bayesian network

Suppose that there are two events which could cause grass to be wet: either the sprinkler is on or it's raining. Also, suppose that the rain has a direct effect on the use of the sprinkler (namely that when it rains, the sprinkler is usually not turned on). Then the situation can be modeled with a Bayesian network (shown to the right). All three variables have two possible values, T (for true) and F (for false).

The joint probability function is:

$P (G,S,R)= P (G| S,R) P(S| R) P(R)$

where the names of the variables have been abbreviated to G = Grass wet (yes/no), S = Sprinkler turned on (yes/no), and R = Raining (yes/no).

![SimpleBayesNet](https://upload.wikimedia.org/wikipedia/commons/0/0e/SimpleBayesNet.svg)

The model can answer questions like "What is the probability that it is raining, given the grass is wet?" by using the conditional probability formula and summing over all nuisance variables:



In [23]:
rain = B(20,100)
sprinkler = Lea.if_(rain, B(1,100), B(40,100))
grassWet = Lea.buildCPT(
    ( ~sprinkler & ~rain, False               ),
    ( ~sprinkler &  rain, B(80,100)),
    (  sprinkler & ~rain, B(90,100)),
    (  sprinkler &  rain, B(99,100))
)

grassWet

False : 27581/50000
 True : 22419/50000

## References

https://bitbucket.org/piedenis/lea

https://en.wikipedia.org/wiki/Bayesian_network